In [28]:
import numpy as np

In [29]:
# Number of desired iterations
N=10**4

# only about 1/6 of random conditions satisfy reasonability requirements
N=6*N

# All values have min 0, adjust values below to change upper bound
nu=10
nd=10
A=100
beta=1 #needs to be  <=1
delta=1 #needs to be <=1
Iu=100
Id=100
I0=100

np.random.seed(314159)

# generate 6Nx8 matrix of random seed evaluation points
randos=np.random.rand(N,8)
# used to normalize values
scalars=np.diag([nu,nd,A,beta,delta,Iu,Id,I0])

#do the normalization
inputs=np.matmul(randos,scalars)
print(inputs.shape)

# Drop points with impossible conditions, nu<nd,Iu<Id,Iu<I0
inputs = inputs[inputs[:,0]>inputs[:,1]]
inputs = inputs[inputs[:,5]>inputs[:,6]]
inputs = inputs[inputs[:,5]>inputs[:,7]]
print(inputs.shape)

(60000, 8)
(9908, 8)


In [30]:
# Create expected utility at parameters

def compute_U(input_row):
  nu,nd,A,beta,delta,Iu,Id,I0=input_row
  Q=(1-delta)*A+I0 #constant to make formulas shorter
  # Compute utilities
  uA=(Q+I0)*nd+beta*((1-delta)*(Q+I0)+2*Id)*nd
  uB=(Q)*nu+beta*((1-delta)*(Q)+2*Iu)*nd
  uC=(Q+I0)*nd+beta*((1-delta)*(Q+I0)+Id)*nu
  uD=(Q)*nu+beta*((1-delta)*Q+Iu)*nu

  #return each utility
  return [uA, uB, uC, uD]

#compute utility for each strategy at every input
U=np.apply_along_axis(compute_U,1,inputs) 
U.shape

(9908, 4)

In [31]:
def rank(LETTERS): #function to define a potential ranking of outcomes
  # accepts a vector [n1,n2,n3,n4] giving which is preferred to which
  TruthTable= \
  (U[:,LETTERS[0]]>U[:,LETTERS[1]])& \
  (U[:,LETTERS[1]]>U[:,LETTERS[2]])& \
  (U[:,LETTERS[2]]>U[:,LETTERS[3]])
  #returns a Nx1 vector of whether or not the proposed ranking is true for the set of inputs
  return TruthTable

import itertools
#makes all permutations, which is all possible rankings
LETTERS=np.array(list(itertools.permutations([0,1,2,3]))) 

In [32]:
# for each possible ranking compute when that's possible
OUT=np.apply_along_axis(rank,1,LETTERS)

In [33]:
# see which rankings occur a non-zero amount of times
#STRATS=np.array(list(itertools.permutations(["low,low>","high,low>","low,high>","high,high>"]))) 
STRATS=np.array(list(itertools.permutations(["A","B","C","D"]))) 
Outcome=np.append(STRATS,np.apply_along_axis(np.count_nonzero,1,OUT).reshape(-1,1),axis=1)
print(Outcome)

[['A' 'B' 'C' 'D' '391']
 ['A' 'B' 'D' 'C' '0']
 ['A' 'C' 'B' 'D' '1041']
 ['A' 'C' 'D' 'B' '0']
 ['A' 'D' 'B' 'C' '0']
 ['A' 'D' 'C' 'B' '0']
 ['B' 'A' 'C' 'D' '380']
 ['B' 'A' 'D' 'C' '307']
 ['B' 'C' 'A' 'D' '82']
 ['B' 'C' 'D' 'A' '56']
 ['B' 'D' 'A' 'C' '754']
 ['B' 'D' 'C' 'A' '678']
 ['C' 'A' 'B' 'D' '309']
 ['C' 'A' 'D' 'B' '57']
 ['C' 'B' 'A' 'D' '40']
 ['C' 'B' 'D' 'A' '26']
 ['C' 'D' 'A' 'B' '43']
 ['C' 'D' 'B' 'A' '106']
 ['D' 'A' 'B' 'C' '0']
 ['D' 'A' 'C' 'B' '0']
 ['D' 'B' 'A' 'C' '0']
 ['D' 'B' 'C' 'A' '4500']
 ['D' 'C' 'A' 'B' '0']
 ['D' 'C' 'B' 'A' '1138']]


In [34]:
# add input data
out_data=np.append(inputs,OUT.T,axis=1)
A_win=np.sum(OUT.T[:,0:6],axis=1)
B_win=np.sum(OUT.T[:,6:12],axis=1)
C_win=np.sum(OUT.T[:,12:18],axis=1)
D_win=np.sum(OUT.T[:,18:24],axis=1)

In [35]:
OUT.shape


(24, 9908)

# Sensitivity Analysis
At this point, we know which strategy is preferred ex ante. We should take their ex ante strategy, apply their optimal period one behavior, add a production shock (the algorithm) and see how the size of the shock changes their strategy.

We can proceed similarly, but not identically. We right now have $4$ vectors that tell which strategy won for each simulation. I will extract the first period behavior $A$ and $C$ have low first periods. $B$ and $D$ have high first periods.

I will recall the parameters for this simulation, compute the deterministic component of audience, then add the shock.

We will then have a new audience size, and I will compare the high quality strategy to the low quality strategy.

In [36]:
low_period_one = A_win+C_win
low_period_one = low_period_one.astype(np.bool)

factors=np.append(inputs,np.reshape(low_period_one,(-1,1)),axis=1)

# accepts a row vector of parameters including whether or not they choose 
# low strat period one
def shocks(factor_row,sigma=1):
  def shock_dist(sigma):
    return(np.random.normal(loc=0,scale=sigma))
  
  nu,nd,A,beta,delta,Iu,Id,I0,low=factor_row
  Q=(1-delta)*A+I0

  if low:
    total_shock=shock_dist(sigma)+shock_dist(sigma)
    aud=Q+I0+total_shock
    u_high = ((1-delta)*aud+Id)*nu # C
    u_low = ((1-delta)*aud+2*Id)*nd # A

    aud_det=Q+I0
    u_high_det = ((1-delta)*aud_det+Id)*nu #Not problematic
    u_low_det = ((1-delta)*aud_det+2*Id)*nd
  else :
    total_shock = shock_dist(sigma)
    aud=Q+total_shock
    u_high = ((1-delta)*aud+Iu)*nu # D
    u_low = ((1-delta)*aud+2*Iu)*nd # B

    aud_det=Q
    u_high_det = ((1-delta)*aud_det+Iu)*nu #Problematic (Q)*nu+beta*((1-delta)*Q+Iu)*nu
    u_low_det = ((1-delta)*aud_det+2*Iu)*nd

  return [u_high,u_low,aud,u_high_det,u_low_det,aud_det,total_shock]



In [37]:
# Do this first to match output
shock_output_100=np.apply_along_axis(shocks,1,factors,sigma=100)
shock_output=shock_output_100

shock_output_0=np.apply_along_axis(shocks,1,factors,sigma=0) 
shock_output_1=np.apply_along_axis(shocks,1,factors,sigma=1) 
shock_output_2=np.apply_along_axis(shocks,1,factors,sigma=3.1622777) 
shock_output_3=np.apply_along_axis(shocks,1,factors,sigma=10) 
shock_output_4=np.apply_along_axis(shocks,1,factors,sigma=31.622777) 
 
shock_output_5=np.apply_along_axis(shocks,1,factors,sigma=316.22777) 
shock_output_6=np.apply_along_axis(shocks,1,factors,sigma=1000) 
shock_output_7=np.apply_along_axis(shocks,1,factors,sigma=3162.2777) 
shock_output_8=np.apply_along_axis(shocks,1,factors,sigma=10000)


#shock_output

In [38]:
def format_shock_output(data,var):
  high_after_shock=data[:,0]>data[:,1]
  high_after_shock=np.reshape(high_after_shock,(-1,1))
  high_ex_ante=data[:,3]>data[:,4]
  high_ex_ante=np.reshape(high_ex_ante,(-1,1))

  diff=high_after_shock!=high_ex_ante

  compare=np.append(high_after_shock,high_ex_ante,axis=1)
  compare=np.append(compare,diff,axis=1)
  var_column=np.ones([compare.shape[0],1])*var
  compare=np.append(compare,var_column,axis=1)
  return compare

compare=format_shock_output(shock_output,100)
compare

array([[  1.,   1.,   0., 100.],
       [  0.,   1.,   1., 100.],
       [  0.,   1.,   1., 100.],
       ...,
       [  1.,   1.,   0., 100.],
       [  0.,   0.,   0., 100.],
       [  1.,   1.,   0., 100.]])

In [39]:
format_shock_output(shock_output_100,100)

array([[  1.,   1.,   0., 100.],
       [  0.,   1.,   1., 100.],
       [  0.,   1.,   1., 100.],
       ...,
       [  1.,   1.,   0., 100.],
       [  0.,   0.,   0., 100.],
       [  1.,   1.,   0., 100.]])

In [40]:
np.mean(compare[:,2])

0.11637060960839725

# Save output data

In [41]:
import pandas as pd 
from google.colab import files

names=np.array(list(itertools.permutations(["A","B","C","D"])))
names=np.apply_along_axis(lambda d: d[0] + d[1] + d[2]+d[3] , 1, names)
colnames=np.append(['nu','nd','iA','beta','delta','Iu','Id','I0'],names)

out_data=pd.DataFrame(out_data,columns=colnames)
out_data['A_win']=A_win
out_data['B_win']=B_win
out_data['C_win']=C_win
out_data['D_win']=D_win

def func(row):
    if row['A_win'] == 1:
        return "A"
    elif row['B_win'] ==1:
        return "B"
    elif row['C_win'] ==1:
        return "C"
    elif row['D_win'] ==1:
        return 'D'

out_data['winner'] = out_data.apply(func, axis=1)


out_data[['u_high','u_low','aud','u_high_det','u_low_det','aud_det','total_shock']]=pd.DataFrame(shock_output)


out_data[['high_after_shock','high_ex_ante','different','var']] = pd.DataFrame(compare)
out_data[['uA','uB','uC','uD']]=U


out_data.to_csv("SimulationDataWithShocks.csv")
files.download("SimulationDataWithShocks.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
U

array([[ 955.41732273,  835.01852569,  961.04038618,  812.2748999 ],
       [1266.76881951, 1151.79247628, 1334.31810339, 1162.57726904],
       [  66.63226449,  192.44727749,   75.73227805,  208.88895137],
       ...,
       [  38.19196692,  396.60086935,  157.36348982,  505.18208622],
       [1348.18719761, 1158.99211025, 1078.75757236,  874.85499094],
       [  29.07983093,   94.63597838,   29.82416235,  110.01362153]])

In [43]:
#shock_output
LETTERS

array([[0, 1, 2, 3],
       [0, 1, 3, 2],
       [0, 2, 1, 3],
       [0, 2, 3, 1],
       [0, 3, 1, 2],
       [0, 3, 2, 1],
       [1, 0, 2, 3],
       [1, 0, 3, 2],
       [1, 2, 0, 3],
       [1, 2, 3, 0],
       [1, 3, 0, 2],
       [1, 3, 2, 0],
       [2, 0, 1, 3],
       [2, 0, 3, 1],
       [2, 1, 0, 3],
       [2, 1, 3, 0],
       [2, 3, 0, 1],
       [2, 3, 1, 0],
       [3, 0, 1, 2],
       [3, 0, 2, 1],
       [3, 1, 0, 2],
       [3, 1, 2, 0],
       [3, 2, 0, 1],
       [3, 2, 1, 0]])

# Varying Variance

In [44]:
names=np.array(list(itertools.permutations(["A","B","C","D"])))
names=np.apply_along_axis(lambda d: d[0] + d[1] + d[2]+d[3] , 1, names)
colnames=np.append(['nu','nd','iA','beta','delta','Iu','Id','I0'],names)

out_data=pd.DataFrame(out_data,columns=colnames)
out_data['A_win']=A_win
out_data['B_win']=B_win
out_data['C_win']=C_win
out_data['D_win']=D_win

def func(row):
    if row['A_win'] == 1:
        return "A"
    elif row['B_win'] ==1:
        return "B"
    elif row['C_win'] ==1:
        return "C"
    elif row['D_win'] ==1:
        return 'D'

out_data['winner'] = out_data.apply(func, axis=1)
out_data[['uA','uB','uC','uD']]=U
out_data = pd.DataFrame(np.repeat(out_data.values, 10, axis=0),columns=out_data.columns)


# shock_output_0=np.apply_along_axis(shocks,1,factors,sigma=0) 
# shock_output_1=np.apply_along_axis(shocks,1,factors,sigma=1) 
# shock_output_2=np.apply_along_axis(shocks,1,factors,sigma=3.1622777) 
# shock_output_3=np.apply_along_axis(shocks,1,factors,sigma=10) 
# shock_output_4=np.apply_along_axis(shocks,1,factors,sigma=31.622777) 
 
# shock_output_5=np.apply_along_axis(shocks,1,factors,sigma=316.22777) 
# shock_output_6=np.apply_along_axis(shocks,1,factors,sigma=1000) 
# shock_output_7=np.apply_along_axis(shocks,1,factors,sigma=3162.2777) 
# shock_output_8=np.apply_along_axis(shocks,1,factors,sigma=10000)

compare_0=format_shock_output(shock_output_0,0)
compare_1=format_shock_output(shock_output_1,1)
compare_2=format_shock_output(shock_output_2,3.1622777)
compare_3=format_shock_output(shock_output_3,10)
compare_4=format_shock_output(shock_output_4,31.622777)
compare_5=format_shock_output(shock_output_5,316.22777)
compare_6=format_shock_output(shock_output_6,1000)
compare_7=format_shock_output(shock_output_7,3162.2777)
compare_8=format_shock_output(shock_output_8,10000)
compare_100=format_shock_output(shock_output_100,100)

big_compare=np.concatenate((compare_0,compare_1,compare_2,compare_3,compare_4,compare_5,compare_6,compare_7,compare_8,compare_100),axis=0)
big_shock_output=np.concatenate((shock_output_0,shock_output_1,shock_output_2,shock_output_3,shock_output_4,shock_output_5,shock_output_6,shock_output_7,shock_output_8,shock_output_100),axis=0)

out_data[['u_high','u_low','aud','u_high_det','u_low_det','aud_det','total_shock']]=pd.DataFrame(big_shock_output)


out_data[['high_after_shock','high_ex_ante','different','var']] = pd.DataFrame(big_compare)


out_data.to_csv("SimulationDataWithShocks_multiple_variance.csv")
files.download("SimulationDataWithShocks_multiple_variance.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
out_data.columns

Index(['nu', 'nd', 'iA', 'beta', 'delta', 'Iu', 'Id', 'I0', 'ABCD', 'ABDC',
       'ACBD', 'ACDB', 'ADBC', 'ADCB', 'BACD', 'BADC', 'BCAD', 'BCDA', 'BDAC',
       'BDCA', 'CABD', 'CADB', 'CBAD', 'CBDA', 'CDAB', 'CDBA', 'DABC', 'DACB',
       'DBAC', 'DBCA', 'DCAB', 'DCBA', 'A_win', 'B_win', 'C_win', 'D_win',
       'winner', 'uA', 'uB', 'uC', 'uD', 'u_high', 'u_low', 'aud',
       'u_high_det', 'u_low_det', 'aud_det', 'total_shock', 'high_after_shock',
       'high_ex_ante', 'different', 'var'],
      dtype='object')